# Botnet flow and profiling

In [96]:
# IMPORTS"
from pprint import pprint
from collections import Counter
import random
import time
from scipy import stats
import pandas as pd

In [108]:
scenario_10 = 'capture20110818.pcap.netflow.labeled'


def parse_line(split_line):
    src = split_l[3].split(':')
    dst = split_l[5].split(':')

    return {
        'datetime': split_l[0],
        'duration': split_l[1],
        'protocol': split_l[2],
        'src_ip': src[0],
        'src_port': '' if len(src) == 1 else src[1],
        'dst_ip': dst[0],
        'dst_port': '' if len(dst) == 1 else src[1],
        'tos': split_l[6],
        'packets': int(split_l[7]),
        'bytes': int(split_l[8]),
        'flows': split_l[9],
        'label': split_l[10]
    }

parsed = []
with open('Data/{}'.format(scenario_10)) as f:                                
    next(f) # Skip header
    for idx, l in enumerate(f):
        try:
            split_l = list(filter(None, l.split('\t')))
            parsed.append(parse_line(l))
        except Exception as e:
#             print('ERROR on line {}, raw string {}, splitted as {}'.format(idx, l, split_l))
            try:
#                 print('Attempting to pars with space as dividing character...')
                split_l = list(filter(None, l.split(' ')))
                split_l[0:2] = [' '.join(split_l[0:2])] # Merge date and time again
                parsed.append(parse_line(l))
            except Exception as e:
                print('Final ERROR, skipping line, splitted as {}, error {}'.format(split_l, e))

In [ ]:
pprint(parsed[-1])

In [111]:
scenario_data = pd.DataFrame.from_records(parsed)
scenario_data['datetime'] = pd.to_datetime(scenario_data['datetime'])
scenario_data.set_index(['datetime'], inplace=True)
scenario_data

,bytes,dst_ip,dst_port,duration,flows,label,packets,protocol,src_ip,src_port,tos
datetime,,,,,,,,,,,
2011-08-18 10:19:13.328,4,212.24.150.110,33426,0.002,321,1,0,TCP,147.32.86.166,33426,FRPA_
2011-08-18 10:19:13.328,617,147.32.84.59,41915,4.995,40095,1,0,UDP,82.39.2.249,41915,INT
2011-08-18 10:19:13.329,1290,82.39.2.249,43087,4.996,1909200,1,0,UDP,147.32.84.59,43087,INT
2011-08-18 10:19:13.330,1,147.32.192.34,42020,0.000,66,1,0,TCP,147.32.86.166,42020,A_
2011-08-18 10:19:13.330,2,147.32.86.166,25443,0.000,169,1,0,TCP,212.24.150.110,25443,FPA_
2011-08-18 10:19:13.333,25,147.32.84.2,49190,4.185,1658,1,0,TCP,115.184.37.24,49190,A_
2011-08-18 10:19:13.335,311,147.32.86.24,51287,4.978,70580,1,0,TCP,80.78.79.156,51287,PA_
2011-08-18 10:19:13.335,292,151.41.188.39,31002,4.978,64319,1,0,UDP,147.32.86.24,31002,INT
2011-08-18 10:19:13.336,1,147.32.84.118,39051,0.000,145,1,0,UDP,82.73.244.56,39051,INT


## Task 3

We aim to learn a sequential model from NetFlow data from an infected host (unidirectional netfows). Consider scenario 10 from the CTU-13 data sets (see paper 4 from below resources). Remove all background flows from the data. You are to discretize the NetFlows. Investigate the data from one of the infected hosts. Select and vizualize two features that you believe are most relevant for modeling the behavior of the infected host. Discretize these features using use any of the methods discussed in class (combine the two values into a single discrete value). Do you observe any behavior in the two features that could be useful for detecting the infection? Explain. Apply the discretization to data from all hosts in the selected scenario.

- Infected hosts
    - 147.32.84.165: Windows XP English version Name: SARUMAN. Label: Botnet. Amount of bidirectional flows: 9579
    - 147.32.84.191: Windows XP English version Name: SARUMAN1. Label: Botnet. Amount of bidirectional flows: 10454
    - 147.32.84.192: Windows XP English version Name: SARUMAN2. Label: Botnet. Amount of bidirectional flows: 10397
    - 147.32.84.193: Windows XP English version Name: SARUMAN3. Label: Botnet. Amount of bidirectional flows: 10009
    - 147.32.84.204: Windows XP English version Name: SARUMAN4. Label: Botnet. Amount of bidirectional flows: 11159
    - 147.32.84.205: Windows XP English version Name: SARUMAN5. Label: Botnet. Amount of bidirectional flows: 11874
    - 147.32.84.206: Windows XP English version Name: SARUMAN6. Label: Botnet. Amount of bidirectional flows: 11287
    - 147.32.84.207: Windows XP English version Name: SARUMAN7. Label: Botnet. Amount of bidirectional flows: 10581
    - 147.32.84.208: Windows XP English version Name: SARUMAN8. Label: Botnet. Amount of bidirectional flows: 11118
    - 147.32.84.209: Windows XP English version Name: SARUMAN9. Label: Botnet. Amount of bidirectional flows: 9894
- Normal hosts:
    - 147.32.84.170 (amount of bidirectional flows: 10216, Label: Normal-V42-Stribrek)
    - 147.32.84.134 (amount of bidirectional flows: 1091, Label: Normal-V42-Jist)
    - 147.32.84.164 (amount of bidirectional flows: 3728, Label: Normal-V42-Grill)
    - 147.32.87.36 (amount of bidirectional flows: 99, Label: CVUT-WebServer. This normal host is not so reliable since is a webserver)
    - 147.32.80.9 (amount of bidirectional flows: 651, Label: CVUT-DNS-Server. This normal host is not so reliable since is a dns server)
    - 147.32.87.11 (amount of bidirectional flows: 4, Label: MatLab-Server. This normal host is not so reliable since is a matlab server)

ToS = Type of Service

#### Infected - from host

In [ ]:
# Get all data from the first infected host

# Infected from Thu Aug 18 11:06:10 CEST 2011
infected_host_ip = '147.32.84.165'

src_infected_host_data = scenario_data.loc[scenario_data['src_ip'] == infected_host_ip]
not_yet_infected = src_infected_host_data.loc['2011-08-18 00:00:00':'2011-08-18 11:06:09']
print(not_yet_infected.shape)
not_yet_infected

In [113]:
infected = src_infected_host_data.loc['2011-08-18 11:06:09':'2011-08-19 00:00:00']
print(infected.shape)
infected

,bytes,dst_ip,dst_port,duration,flows,label,packets,protocol,src_ip,src_port,tos
datetime,,,,,,,,,,,
2011-08-18 11:06:25.671,1,147.32.80.9,1025,0.000,76,1,0,UDP,147.32.84.165,1025,INT
2011-08-18 11:06:25.673,10,78.40.125.4,1039,3.778,863,1,0,TCP,147.32.84.165,1039,SPA_
2011-08-18 11:06:25.734,3,78.40.125.4,113,0.061,239,1,0,TCP,147.32.84.165,113,SPA_
2011-08-18 11:06:25.823,1,85.190.0.3,8000,0.000,60,1,0,TCP,147.32.84.165,8000,RA_
2011-08-18 11:06:25.823,1,85.190.0.3,3128,0.000,60,1,0,TCP,147.32.84.165,3128,RA_
2011-08-18 11:06:25.823,1,85.190.0.3,3128,0.000,60,1,0,TCP,147.32.84.165,3128,RA_
2011-08-18 11:06:25.823,1,85.190.0.3,8080,0.000,60,1,0,TCP,147.32.84.165,8080,RA_
2011-08-18 11:06:25.823,1,85.190.0.3,8000,0.000,60,1,0,TCP,147.32.84.165,8000,RA_
2011-08-18 11:06:25.824,1,85.190.0.3,80,0.000,60,1,0,TCP,147.32.84.165,80,RA_


#### Compare - from host

In [114]:
print('Counts infected ToS:')
print(infected['tos'].value_counts())

print('\n')

print('Counts clean ToS:')
print(not_yet_infected['tos'].value_counts())

Counts infected ToS:
ECO      8370
UNK      8098
ROB       358
INT       332
RA_       276
RTA       257
TST       256
RTS       255
MAS       255
IRQ       254
___       133
PA_        81
SRC        59
IAH        58
PAR        56
MRQ        48
UR         47
WAY        44
SKP        44
NNA        43
PHO        42
RED        40
TRC        39
AHA        36
MRP        36
PTB        34
DNP        34
SEC        33
NRA        33
NNS        32
MHR        32
DNQ        31
NRS        28
A_         27
DCE        26
TXD        26
SPA_        7
SA_         4
SRPA_       2
URHU        1
Name: tos, dtype: int64


Counts clean ToS:
INT      10
RA_       9
SRPA_     2
SPA_      1
Name: tos, dtype: int64


### Infected - to host

In [23]:
src_infected_host_data = scenario_data.loc[scenario_data['dst_ip'] == infected_host_ip]
src_infected_host_data

,bytes,datetime,dst_ip,dst_port,duration,flows,label,packets,protocol,src_ip,src_port,tos
452,1,2011-08-18 10:39:50.938,147.32.84.165,6667,0.000,273,1,0,TCP,213.232.93.3,6667,FPA_
69882,2,2011-08-18 10:49:11.652,147.32.84.165,51000,0.000,128,1,0,TCP,85.190.0.3,51000,S_RA
69883,2,2011-08-18 10:49:11.652,147.32.84.165,38519,0.000,128,1,0,TCP,85.190.0.3,38519,S_RA
69884,2,2011-08-18 10:49:11.652,147.32.84.165,34349,0.000,128,1,0,TCP,85.190.0.3,34349,S_RA
69885,2,2011-08-18 10:49:11.652,147.32.84.165,49044,0.000,128,1,0,TCP,85.190.0.3,49044,S_RA
69886,2,2011-08-18 10:49:11.652,147.32.84.165,43333,0.000,128,1,0,TCP,85.190.0.3,43333,S_RA
69887,2,2011-08-18 10:49:11.652,147.32.84.165,59128,0.000,128,1,0,TCP,85.190.0.3,59128,S_RA
69888,2,2011-08-18 10:49:11.652,147.32.84.165,45857,0.000,128,1,0,TCP,85.190.0.3,45857,S_RA
69889,2,2011-08-18 10:49:11.652,147.32.84.165,39126,0.000,128,1,0,TCP,85.190.0.3,39126,S_RA
69890,2,2011-08-18 10:49:11.652,147.32.84.165,51071,0.000,128,1,0,TCP,85.190.0.3,51071,S_RA
